In [22]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StandardScalerModel
from pyspark.ml.feature import StandardScaler

In [23]:
spark=SparkSession.builder.appName("KMeans").getOrCreate()

In [24]:
df=spark.read.csv("seeds_dataset.csv",inferSchema=True,header=True)

In [25]:
feature_list=['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [26]:
assembler=VectorAssembler(inputCols=feature_list,outputCol="featureVector")

In [27]:
scaler=StandardScaler(inputCol="featureVector",outputCol="scaledFeature")

In [28]:
df=assembler.transform(df)

In [29]:
df=scaler.fit(df).transform(df)

In [30]:
df.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)
 |-- featureVector: vector (nullable = true)
 |-- scaledFeature: vector (nullable = true)



In [47]:
kmeans=KMeans(featuresCol="scaledFeature",k=3)

In [48]:
model=kmeans.fit(df)

In [49]:
result=model.transform(df)

In [50]:
result.show()

+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+----------+
| area|perimeter|compactness|  length_of_kernel|   width_of_kernel|asymmetry_coefficient|  length_of_groove|       featureVector|       scaledFeature|prediction|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+----------+
|15.26|    14.84|      0.871|             5.763|             3.312|                2.221|              5.22|[15.26,14.84,0.87...|[5.24452795332028...|         0|
|14.88|    14.57|     0.8811| 5.553999999999999|             3.333|                1.018|             4.956|[14.88,14.57,0.88...|[5.11393027165175...|         0|
|14.29|    14.09|      0.905|             5.291|3.3369999999999997|                2.699|             4.825|[14.29,14.09,0.90...|[4.91116018695588...|         0|
|13.84|    13.94|     0.8955

In [51]:
model.computeCost(result)

428.60820118716356